# Distribución de cargas simulado con algoritmo genetico con número enteros

## Pasos
- Dividir la carga entre los procesadores y asignar la cantidad restante
- Establecer la ventana, también conocida cómo la cantidad de procesos
- Asignar las cargas
- Identificar el procesador más cargado y su número de cargas

In [69]:
import random

CE = "    "

In [70]:
class DivisionCargasAGInt:
    def __init__(self,procesos:list, num_procesadores: int, underload: int, overload: int, size_window:int,  
                 rango_valores:list, num_cromosomas:int, num_generaciones=200, prob_cruce=0.8, prob_mutacion=0.05, porcentaje_elitismo=0.05):
        # Algoritmo genetico
        self.rango_valores = rango_valores
        self.num_cromosomas = num_cromosomas
        self.num_generaciones = num_generaciones
        self.prob_cruce = prob_cruce
        self.prob_mutacion = prob_mutacion
        self.porcentaje_elitismo = porcentaje_elitismo

        # Distribución de cargas
        self.num_procesadores = num_procesadores
        self.underload = underload
        self.overload = overload
        self.procesos = procesos
        self.size_window = size_window
        self.solucion = []
        self.carga_maxima = None

    
    def crear_cromosoma(self):
        cromosoma = self.rango_valores.copy()
        random.shuffle(cromosoma)
        return cromosoma
    
    def crear_poblacion(self):
        return [self.crear_cromosoma() for _ in range(self.num_cromosomas)]

    def seleccion(self, cromosomas:list):
        return random.choice(cromosomas)
    
    def getElite(self, poblacion: list, ventana:list):
        num_elitismo = int(self.num_cromosomas * self.porcentaje_elitismo)
        return sorted(poblacion, key=lambda c: self.evaluar_fitness(c,ventana), reverse=True)[:num_elitismo]
    
    def best(self, poblacion, ventana):
        return max(poblacion, key=lambda c: self.evaluar_fitness(c, ventana))

    def fit(self):
        for i in range(0,len(self.procesos), self.size_window):
            ventana = self.procesos[i:i+self.size_window]

            if len(ventana) < self.num_procesadores:
                ceros = self.num_procesadores - len(ventana)
                ventana += [0] * ceros
            
            poblacion = self.crear_poblacion()
            for _ in range(self.num_generaciones):
                poblacion = self.newPoblacion(poblacion, ventana)
            
            mejor = self.best(poblacion, ventana)
            self.solucion.append( (mejor, self.evaluar_fitness(mejor, ventana)) )

    
    def newPoblacion(self, poblacion:list, ventana:list):
        nueva_poblacion = []
        nueva_poblacion.extend(self.getElite(poblacion, ventana))

        while (len(nueva_poblacion) + 2) <= (self.num_cromosomas):

            if random.random() <= self.prob_cruce: 
                padre1 = self.seleccion(poblacion)
                padre2 = self.seleccion(poblacion)
                
                hijo1, hijo2 = self.cruzar(padre1, padre2)

                nueva_poblacion.extend([hijo1, hijo2])

        return nueva_poblacion
    
    def cruzar(self, padre1: list, padre2: list):
        L = len(padre1)
        H1 = [None] * L
        H2 = [None] * L

        l1 = random.randint(0, L - 1)
        l2 = random.randint(l1 + 1, L)

        # Llenamos los hijos con los valores de los intervalos definidos del padre contrario
        H1[l1:l2] = padre2[l1:l2]
        H2[l1:l2] = padre1[l1:l2]
        # print(f"{H1}\n{H2}\n")

        for h in range(2):
            if h == 0:
                Ha = H1
                Pa = padre1
                Pc = padre2
            else:
                Ha = H2
                Pa = padre2
                Pc = padre1

            # Completamos con los valores del mismo padre que no se encuentren en el hijo
            for i in range(len(Ha)):
                if (Ha[i] == None):
                    if (Pa[i] not in Ha):
                        Ha[i] = Pa[i]
            
            # Terminamos completando al hijo con los valores del padre contrario que no se encuentren
            #  en el orden de izquierda a derecha
            for i in range(len(Pc)):
                if (Pc[i] not in Ha):
                    Ha[Ha.index(None)] = Pc[i]
            
            if h == 0:
                H1 = Ha
            else:
                H2 = Ha

        # print(f"{H1}\n{H2}")
        # print(len(set(H1)) == len(H1) and len(set(H2)) == len(H2)) # Comprobar que no hay elementos repetidos en ambas listas, todo se hizo correctamente
        
        return H1, H2

    def evaluar_fitness(self, cromosoma:list, ventana:list):
        carga_por_procesador = [0] * self.num_procesadores
        
        #[5,4,3,5,4,3] ventana
        #[2,0,1,3,4,5] cromosoma

        i = 0
        for asignacion in cromosoma:
            if i < self.num_procesadores:
                carga_por_procesador[i] += ventana[asignacion]
                i += 1
            else:
                i = 0
                carga_por_procesador[i] += ventana[asignacion]
                i += 1

        self.carga_maxima = max(carga_por_procesador)
        cargas_aceptadas = sum(1 for carga in carga_por_procesador if self.underload < carga < self.overload)
        promedio_carga = sum(carga / self.carga_maxima for carga in carga_por_procesador) / self.num_procesadores

        return (1.0 / self.carga_maxima) * promedio_carga * (cargas_aceptadas / self.num_procesadores)

## Parametros

In [71]:
n_procesos = 3600
n_procesadores = 4
overloaded = (n_procesos // n_procesadores) + 100
size_window = 6
rango_valores = [i for i in range(n_procesadores)]

## Implementacion

In [72]:
procesos = [184, 158, 119, 172, 242, 203, 61, 200, 239, 69, 248, 128, 119, 75, 59, 215, 332, 296, 215, 36, 230]

division_cargas_ag = DivisionCargasAGInt(
    procesos = procesos,
    num_procesadores = n_procesadores, 
    size_window=size_window,
    underload = 50, overload = overloaded, 
    num_cromosomas = 100, rango_valores=rango_valores)

division_cargas_ag.fit()
best = division_cargas_ag.solucion

print("Carga de procesos:", procesos)
print("\nLa mejor division de cargas es:")
for i, slide in enumerate(best, start=1):
    print(f"{CE}- Slide {i}: {slide[0]}")
    print(f"{CE * 2}- Fitness: {slide[1]}")

Carga de procesos: [184, 158, 119, 172, 242, 203, 61, 200, 239, 69, 248, 128, 119, 75, 59, 215, 332, 296, 215, 36, 230]

La mejor division de cargas es:
    - Slide 1: [2, 1, 0, 3]
        - Fitness: 0.004674208412098299
    - Slide 2: [0, 2, 3, 1]
        - Fitness: 0.002490327550287985
    - Slide 3: [0, 2, 1, 3]
        - Fitness: 0.002531097890751758
    - Slide 4: [0, 2, 3, 1]
        - Fitness: 0.001136578449905482
